In [6]:
library(tidyverse)
library(caret)
library(glmnet)
library(dplyr)
#library(psych) 
library(lme4)
library(e1071)
library(GGally)
options(warn=-1)
options(scipen=999)
#library(crosstable)

In [7]:
seed_list<-c(1001,1111,1221,1331,1441,1551,1661,1771,1881,1991)
cut_off<-c(0,10,50,100,500,1000)
time_off<-c(10000,10,20,50,100)
#time_off<-c(10)

lambda <- 10^seq(-3, 3, length = 100)
alpha<-seq(0,1,length=10)

In [8]:
library(doParallel)
cl <- makePSOCKcluster(30)
registerDoParallel(cl)

In [9]:
m<-1
v<-1
#list_of_vi <- vector(mode="list", length=5)
#names(list_of_vi) <- c('10000', '10', '20', '50', '100')

list_of_vi_10000<-list()
list_of_vi_10<-list()
list_of_vi_20<-list()
list_of_vi_50<-list()
list_of_vi_100<-list()

corpus_list<-c("google")
list_of_rsqr=list()
ratings_list<-c("reddy","cordeiro90","cordeiro100")
tagged_list<-c("UnTagged")

ppmi_setting_list<-c("RAW")
comp_setting_list<-c("Aware","Agnostic")
feature_setting_list<-c("withSetting")
caret_spearman <- function(data, lev = NULL, model = NULL) {
  spearman_val <- cor(x = data$pred, y = data$obs, method = "spearman")
  c(Spearman = spearman_val)
}

In [10]:
for (c in corpus_list){
  for (t in tagged_list) {
    for (f in feature_setting_list) {
      for (p in ppmi_setting_list){
        for (a in comp_setting_list){
          for (i in time_off) {
            
            for (j in cut_off) {
              
              
              print(paste0(c," ",p," ",a," ",f," ",t," ",i," ",j))
              
              na_df<-read.csv(paste0("/data/dharp/compounds/datasets/",c,"/features_Compound",a,"_",f,"_",p,"_",t,"_",i,"_",j,"_na.csv"),sep = '\t')
              nona_df<-read.csv(paste0("/data/dharp/compounds/datasets/",c,"/features_Compound",a,"_",f,"_",p,"_",t,"_",i,"_",j,"_med.csv"),sep = '\t')
              
              for (r in ratings_list){
                na_df<-na_df %>% filter(source==r)
                nona_df<-nona_df %>% filter(source==r)
                
                trainX_na<-na_df %>% select(-c(compositionality,modifier,head,avgModifier,stdevModifier,avgHead,stdevHead,stdevHeadModifier,compound,is_adj,source)) %>% select(-one_of("comp_freq_bins"))
                trainY_na<-na_df %>% select(compositionality)
                trainY_na<-trainY_na$compositionality
                
                
                trainX_nona<-nona_df %>% select(-c(compositionality,modifier,head,avgModifier,stdevModifier,avgHead,stdevHead,stdevHeadModifier,compound,is_adj,source)) %>% select(-one_of("comp_freq_bins"))
                trainY_nona<-nona_df %>% select(compositionality)
                trainY_nona<-trainY_nona$compositionality
                
                if (dim(trainX_nona)[1]<10) {
                  break
                }
                for (s in seed_list)  {
                  set.seed(s)
                  seeds <- vector(mode = "list", length = 14)
                  for(z in 1:13) seeds[[z]] <- sample.int(n=1000, 10)
                  #for the last model
                  seeds[[14]]<-sample.int(1000, 1)
                  
                  ridge_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                      trControl = trainControl("cv",number = 10, search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                      preProc = c("nzv","center", "scale")
                  )
                  
                  elastic_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                        trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                        preProc = c("nzv","center", "scale")
                  )
                  
                  lasso_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                      trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                      preProc = c("nzv","center", "scale")
                  )
                  
                  
                  ridge_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                    trControl = trainControl("cv",number = 10, search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                    preProcess = c("nzv","medianImpute", "center", "scale")
                  )
                  
                  elastic_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                      trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                      preProcess = c("nzv","medianImpute", "center", "scale")
                  )
                  
                  lasso_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                    trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                    preProcess = c("nzv","medianImpute", "center", "scale")
                  )
                  
                  
                  ridge_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                               trControl = trainControl("cv",number = 10, search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                               preProc = c("nzv","center", "scale")
                  )
                  
                  elastic_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                                 trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                                 preProc = c("nzv","center", "scale")
                  )
                  
                  lasso_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                               trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                               preProc = c("nzv","center", "scale")
                  )
                  
                  
                  ridge_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                             trControl = trainControl("cv",number = 10, search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                             preProcess = c("nzv","medianImpute", "center", "scale")
                  )
                  
                  elastic_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                               trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                               preProcess = c("nzv","medianImpute", "center", "scale")
                  )
                  
                  lasso_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                             trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                             preProcess = c("nzv","medianImpute", "center", "scale")
                  )
                  
                  
                  perf_elastic_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(elastic_na)[,"method"],TrainRsquared=getTrainPerf(elastic_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(elastic_na_spearman)[,"TrainSpearman"])
                  perf_ridge_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(ridge_na)[,"method"],TrainRsquared=getTrainPerf(ridge_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(ridge_na_spearman)[,"TrainSpearman"])
                  perf_lasso_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(lasso_na)[,"method"],TrainRsquared=getTrainPerf(lasso_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(lasso_na_spearman)[,"TrainSpearman"])
                  
                  
                  perf_elastic_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(elastic_nona)[,"method"],TrainRsquared=getTrainPerf(elastic_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(elastic_nona_spearman)[,"TrainSpearman"])
                  perf_ridge_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(ridge_nona)[,"method"],TrainRsquared=getTrainPerf(ridge_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(ridge_nona_spearman)[,"TrainSpearman"])
                  perf_lasso_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(lasso_nona)[,"method"],TrainRsquared=getTrainPerf(lasso_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(lasso_nona_spearman)[,"TrainSpearman"])
                  
                  varimp_elastic_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(elastic_na)$importance))
                  varimp_ridge_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(ridge_na)$importance))
                  varimp_lasso_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(lasso_na)$importance))
                  
                  varimp_elastic_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(elastic_nona)$importance))
                  varimp_ridge_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(ridge_nona)$importance))
                  varimp_lasso_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(lasso_nona)$importance))    
                  
                  
                  
                  list_of_rsqr[[m]]<-perf_elastic_na
                  m<-m+1
                  
                  list_of_rsqr[[m]]<-perf_elastic_nona
                  m<-m+1
                  
                  list_of_rsqr[[m]]<-perf_ridge_na
                  m<-m+1
                  
                  list_of_rsqr[[m]]<-perf_ridge_nona
                  m<-m+1
                  
                  list_of_rsqr[[m]]<-perf_lasso_na
                  m<-m+1 
                  
                  list_of_rsqr[[m]]<-perf_lasso_nona
                  m<-m+1 
                  
                  if (i==10000) {
                    list_of_vi_10000[[v]]<-varimp_elastic_na
                    v<-v+1
                    list_of_vi_10000[[v]]<-varimp_elastic_nona
                    v<-v+1
                    list_of_vi_10000[[v]]<-varimp_ridge_na
                    v<-v+1
                    list_of_vi_10000[[v]]<-varimp_ridge_nona
                    v<-v+1
                    list_of_vi_10000[[v]]<-varimp_lasso_na
                    v<-v+1
                    list_of_vi_10000[[v]]<-varimp_lasso_nona
                    v<-v+1
                    
                  }
                  else if (i==10) {
                    list_of_vi_10[[v]]<-varimp_elastic_na
                    v<-v+1
                    list_of_vi_10[[v]]<-varimp_elastic_nona
                    v<-v+1
                    list_of_vi_10[[v]]<-varimp_ridge_na
                    v<-v+1
                    list_of_vi_10[[v]]<-varimp_ridge_nona
                    v<-v+1
                    list_of_vi_10[[v]]<-varimp_lasso_na
                    v<-v+1
                    list_of_vi_10[[v]]<-varimp_lasso_nona
                    v<-v+1
                  }
                  
                  else if (i==20) {
                    list_of_vi_20[[v]]<-varimp_elastic_na
                    v<-v+1
                    list_of_vi_20[[v]]<-varimp_elastic_nona
                    v<-v+1
                    list_of_vi_20[[v]]<-varimp_ridge_na
                    v<-v+1
                    list_of_vi_20[[v]]<-varimp_ridge_nona
                    v<-v+1
                    list_of_vi_20[[v]]<-varimp_lasso_na
                    v<-v+1
                    list_of_vi_20[[v]]<-varimp_lasso_nona
                    v<-v+1
                  }
                  else if (i==50) {
                    list_of_vi_50[[v]]<-varimp_elastic_na
                    v<-v+1
                    list_of_vi_50[[v]]<-varimp_elastic_nona
                    v<-v+1
                    list_of_vi_50[[v]]<-varimp_ridge_na
                    v<-v+1
                    list_of_vi_50[[v]]<-varimp_ridge_nona
                    v<-v+1
                    list_of_vi_50[[v]]<-varimp_lasso_na
                    v<-v+1
                    list_of_vi_50[[v]]<-varimp_lasso_nona
                    v<-v+1
                  }
                  else if (i==100) {
                    list_of_vi_100[[v]]<-varimp_elastic_na
                    v<-v+1
                    list_of_vi_100[[v]]<-varimp_elastic_nona
                    v<-v+1
                    list_of_vi_100[[v]]<-varimp_ridge_na
                    v<-v+1
                    list_of_vi_100[[v]]<-varimp_ridge_nona
                    v<-v+1
                    list_of_vi_100[[v]]<-varimp_lasso_na
                    v<-v+1
                    list_of_vi_100[[v]]<-varimp_lasso_nona
                    v<-v+1
                  }
                  
                }
                
              }
            }
          }
        }
      }
    }
  }
}

[1] "google RAW Aware withSetting UnTagged 10000 0"
[1] "google RAW Aware withSetting UnTagged 10000 10"
[1] "google RAW Aware withSetting UnTagged 10000 50"
[1] "google RAW Aware withSetting UnTagged 10000 100"
[1] "google RAW Aware withSetting UnTagged 10000 500"
[1] "google RAW Aware withSetting UnTagged 10000 1000"
[1] "google RAW Aware withSetting UnTagged 10 0"
[1] "google RAW Aware withSetting UnTagged 10 10"
[1] "google RAW Aware withSetting UnTagged 10 50"
[1] "google RAW Aware withSetting UnTagged 10 100"
[1] "google RAW Aware withSetting UnTagged 10 500"
[1] "google RAW Aware withSetting UnTagged 10 1000"
[1] "google RAW Aware withSetting UnTagged 20 0"
[1] "google RAW Aware withSetting UnTagged 20 10"
[1] "google RAW Aware withSetting UnTagged 20 50"
[1] "google RAW Aware withSetting UnTagged 20 100"
[1] "google RAW Aware withSetting UnTagged 20 500"
[1] "google RAW Aware withSetting UnTagged 20 1000"
[1] "google RAW Aware withSetting UnTagged 50 0"
[1] "google RAW Aware wi

In [11]:
rsquared_df<-bind_rows(list_of_rsqr)
rsquared_df$cutoff<-as.factor(rsquared_df$cutoff)
statistics<-rsquared_df %>% group_by(impute,setting,dataset,ml_algo,timespan,cutoff) %>% 
summarise(n=mean(n),meanRsquared=mean(TrainRsquared),sdRsquared=sd(TrainRsquared),meanCorr=mean(TrainSpearman),sdCorr=sd(TrainSpearman))

statistics %>% arrange(desc(meanCorr)) %>% head()

`summarise()` has grouped output by 'impute', 'setting', 'dataset', 'ml_algo', 'timespan'. You can override using the `.groups` argument.


impute,setting,dataset,ml_algo,timespan,cutoff,n,meanRsquared,sdRsquared,meanCorr,sdCorr
<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
python,Agnostic,reddy,elastic,10000,1000,91,0.5645837,0.02434222,0.6982440,0.02267834
python,Aware,reddy,elastic,10000,1000,91,0.5645837,0.02434222,0.6982440,0.02267834
python,Aware,reddy,elastic,100,0,91,0.5553580,0.02341143,0.6982433,0.02740960
python,Agnostic,reddy,elastic,10000,500,91,0.5640398,0.02438456,0.6965973,0.02514993
python,Aware,reddy,elastic,10000,500,91,0.5640398,0.02438456,0.6965973,0.02514993
python,Aware,reddy,elastic,100,10,91,0.5486412,0.02079302,0.6965944,0.02692879


In [12]:
varimp_df<-bind_rows(list_of_vi_10)
varimp_df$cutoff<-as.factor(varimp_df$cutoff)
varimp_df[is.na(varimp_df)] <- 0
varimp_df

,feature,impute,tag,setting,dataset,corpus,ml_algo,n,seed,timespan,⋯,sim_with_modifier.1920,sim_with_modifier.1930,sim_with_modifier.1940,sim_with_modifier.1950,sim_with_modifier.1960,sim_with_modifier.1970,sim_with_modifier.1980,sim_with_modifier.1990,sim_with_modifier.2000,sim_with_modifier.2010
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Overall...1,woSetting,r,UnTagged,Aware,reddy,google,elastic,91,1001,10,⋯,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000
Overall...2,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1001,10,⋯,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000
Overall...3,woSetting,r,UnTagged,Aware,reddy,google,rigde,91,1001,10,⋯,24.86195,19.95579,11.346473,14.96811,17.094141,12.243730,45.31360,32.89011,12.49282,10.57059
Overall...4,woSetting,python,UnTagged,Aware,reddy,google,rigde,91,1001,10,⋯,20.15558,16.75861,6.344700,16.01386,10.455558,8.793591,42.60235,31.68070,15.38507,15.59547
Overall...5,woSetting,r,UnTagged,Aware,reddy,google,lasso,91,1001,10,⋯,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000
Overall...6,woSetting,python,UnTagged,Aware,reddy,google,lasso,91,1001,10,⋯,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000
Overall...7,woSetting,r,UnTagged,Aware,reddy,google,elastic,91,1111,10,⋯,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000
Overall...8,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1111,10,⋯,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000
Overall...9,woSetting,r,UnTagged,Aware,reddy,google,rigde,91,1111,10,⋯,26.70363,23.80994,18.109045,22.48772,24.235693,24.867624,54.68009,44.08311,23.37323,19.96063


In [6]:
rsquared_df<-bind_rows(list_of_rsqr)
rsquared_df$cutoff<-as.factor(rsquared_df$cutoff)
rsquared_df[order(-rsquared_df$TrainSpearman), ]

,feature,impute,tag,setting,dataset,corpus,ml_algo,n,seed,timespan,cutoff,ppmi,method,TrainRsquared,TrainSpearman
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<fct>,<chr>,<chr>,<dbl>,<dbl>
691,woSetting,r,UnTagged,Aware,reddy,google,elastic,91,1551,20,100,RAW,glmnet,0.6655585,0.8095238
693,woSetting,r,UnTagged,Aware,reddy,google,rigde,91,1551,20,100,RAW,glmnet,0.6655585,0.8095238
631,woSetting,r,UnTagged,Aware,reddy,google,elastic,91,1551,20,50,RAW,glmnet,0.6326263,0.7619048
633,woSetting,r,UnTagged,Aware,reddy,google,rigde,91,1551,20,50,RAW,glmnet,0.6326263,0.7619048
446,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1441,10,100,RAW,glmnet,0.5990833,0.7374026
14,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1221,10000,0,RAW,glmnet,0.5759501,0.7341080
74,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1221,10000,10,RAW,glmnet,0.5759430,0.7341080
734,woSetting,python,UnTagged,Agnostic,reddy,google,elastic,91,1221,10000,0,RAW,glmnet,0.5759501,0.7341080
794,woSetting,python,UnTagged,Agnostic,reddy,google,elastic,91,1221,10000,10,RAW,glmnet,0.5759430,0.7341080


In [7]:
rsquared_df[order(-rsquared_df$TrainRsquared), ]

,feature,impute,tag,setting,dataset,corpus,ml_algo,n,seed,timespan,cutoff,ppmi,method,TrainRsquared,TrainSpearman
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<fct>,<chr>,<chr>,<dbl>,<dbl>
691,woSetting,r,UnTagged,Aware,reddy,google,elastic,91,1551,20,100,RAW,glmnet,0.6655585,0.8095238
693,woSetting,r,UnTagged,Aware,reddy,google,rigde,91,1551,20,100,RAW,glmnet,0.6655585,0.8095238
331,woSetting,r,UnTagged,Aware,reddy,google,elastic,91,1551,10,10,RAW,glmnet,0.6386772,0.6904762
333,woSetting,r,UnTagged,Aware,reddy,google,rigde,91,1551,10,10,RAW,glmnet,0.6386772,0.6904762
631,woSetting,r,UnTagged,Aware,reddy,google,elastic,91,1551,20,50,RAW,glmnet,0.6326263,0.7619048
633,woSetting,r,UnTagged,Aware,reddy,google,rigde,91,1551,20,50,RAW,glmnet,0.6326263,0.7619048
446,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1441,10,100,RAW,glmnet,0.5990833,0.7374026
440,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1331,10,100,RAW,glmnet,0.5969103,0.7096666
380,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1331,10,50,RAW,glmnet,0.5954672,0.7081262


In [8]:
max(rsquared_df$TrainRsquared)

[1] 0.5605641

In [18]:
write.csv(rsquared_df,"rsquared_Tagged_PPMI_Agnostic.csv",row.names = FALSE)

In [19]:
write.csv(varimp_df,"varimp_Tagged_PPMI_Agnostic.csv",row.names = FALSE)